<a href="https://www.kaggle.com/code/manavmaheshsanger/fine-tuning-gpt-2-to-recognize-tweet-sentiments?scriptVersionId=211387907" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install datasets pandas transformers evaluate numpy torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00


In [2]:
# Loading and analyzing dataset
from datasets import load_dataset
dataset = load_dataset("mteb/tweet_sentiment_extraction")

README.md:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/3.63M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/465k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/27481 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 3534
    })
})

In [4]:
dataset["train"]

Dataset({
    features: ['id', 'text', 'label', 'label_text'],
    num_rows: 27481
})

In [5]:
import pandas as pd
df = pd.DataFrame(dataset["train"])
df.sample(10)

,id,text,label,label_text
6652,bee1bbb3b9,_EN we`re waiting you here in Belgrade!! Can`...,1,neutral
21043,6ae26d3934,Urgh.... feeling like **** today. Bad headache...,0,negative
25098,4c5caae011,"Watched Adaptation, Interiors, and The Women (...",2,positive
20245,645849b925,Just sittin here waitin for my coffee to be fu...,1,neutral
14180,a146ea7fda,i cant believe this woman talked me into getti...,1,neutral
21664,851583e0de,"http://twitpic.com/4w8l1 - Haaha, my bangs are...",1,neutral
5892,0736535e65,for me they`re not,1,neutral
4407,3fb8232118,Yeah the spammers are discriminating: none o...,0,negative
13134,ba29930e58,playing machines in actinggg,1,neutral
3134,57f97a9ac7,"dont worry, i will. i HATE it when nobody com...",0,negative


In [6]:
df.columns

Index(['id', 'text', 'label', 'label_text'], dtype='object')

## Tokenization

In [7]:
from transformers import GPT2Tokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [8]:
# Initializing the gpt-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

'<|endoftext|>'

In [9]:
def tokenize(element):
    return tokenizer(element["text"], padding="max_length", truncation=True)


tokenized_data = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/27481 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [10]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 3534
    })
})

In [11]:
tokenized_data["train"]["input_ids"][0][:5]

[314, 63, 67, 423, 7082]

In [12]:
tokenized_data["train"]["attention_mask"][0][:5]

[1, 1, 1, 1, 1]

In [13]:
# extracting train and eval sets from tokenized_data, which is a Dataset dictionaery
# trimming the dataset to 1000 because of performance limitations

tokenized_train_dataset = tokenized_data["train"].shuffle(seed=42).select(range(1000))
tokenized_eval_dataset = tokenized_data["test"].shuffle(seed=42).select(range(1000))

## Initializing the model

In [14]:
# Number of labels in our classifier
num_labels = len(df["label"].unique())
num_labels

3

In [15]:
from transformers import GPT2ForSequenceClassification

# Loading GPT-2 model for fine tuning
# number of labels = number of emotions in our dataset

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels = num_labels)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=3, bias=False)
)

## Creating Evaluation method to pass on to trainer

In [17]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

## Fine tuning

In [18]:
from transformers import TrainingArguments, Trainer

In [23]:
# Configuring the trainer
training_args = TrainingArguments(
   output_dir="test_trainer",
   #evaluation_strategy="epoch",
   per_device_train_batch_size=1,  # Reduce batch size here
   per_device_eval_batch_size=1,    # Optionally, reduce for evaluation as well
   gradient_accumulation_steps=4,
    report_to="none"
   )


trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_eval_dataset,
   compute_metrics=compute_metrics

)

In [24]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


TrainOutput(global_step=375, training_loss=0.9870291341145834, metrics={'train_runtime': 556.4018, 'train_samples_per_second': 5.392, 'train_steps_per_second': 0.674, 'total_flos': 1567794659328000.0, 'train_loss': 0.9870291341145834, 'epoch': 3.0})

## Model evaluation

In [25]:
import evaluate
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.7869201898574829,
 'eval_accuracy': 0.697,
 'eval_runtime': 63.3187,
 'eval_samples_per_second': 15.793,
 'eval_steps_per_second': 7.897,
 'epoch': 3.0}

In [26]:
# inferencing from the fine-tuned model
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# specify model storage
model_path = "/kaggle/working/test_trainer/checkpoint-375"

# load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [31]:
def predict(tweet):
    # printing the tweet
    print("Tweet: ", tweet)

    # tokinze the input
    inputs = tokenizer(tweet, return_tensors="pt", truncation=True)

    # get predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # get the logits
    logits = outputs.logits

    # convert logits to probabilities
    probabilities = torch.softmax(logits, dim=-1)

    # get predicted class
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    print("Predicted class: ", predicted_class)

    # Define the label mapping
    label_mapping = {0: "Negative",
                    1: "Neutral",
                    2: "Positive"}

    # get human readable label
    sentiment = label_mapping[predicted_class]

    print(f"Sentiment: {sentiment}", end="\n\n")

In [32]:
# preparing data for evaluation
all_tweets = tokenized_eval_dataset["text"]

import random
tweets = random.sample(all_tweets, 5)
[predict(tweet) for tweet in tweets]

Tweet:  Why is looking for girls names easier than looking for boys
Predicted class:  1
Sentiment: Neutral

Tweet:  I`m a senior. I should of been already.
Predicted class:  1
Sentiment: Neutral

Tweet:   ooh, work im afarid  looking forward to a sunny weekeknd tho!!!
Predicted class:  2
Sentiment: Positive

Tweet:   yep  but I`m going better now
Predicted class:  0
Sentiment: Negative

Tweet:  The Heater blew up
Predicted class:  0
Sentiment: Negative



[None, None, None, None, None]

In [39]:
predict("ohh man")

Tweet:  ohh man
Predicted class:  1
Sentiment: Neutral

